## **Thermodynamic Limits of Atmospheric Water Harvesting**: Supplemental Code
Akshay K. Rao, Andrew J. Fix, David M. Warsinger

---

This notebook uses the "CoolProp" Python wrapper to compute humid air energetics calculations. The water equation of state is based on the IAPWS Formulation. 
1. Bell, I. H., Wronski, J., Quoilin, S. & Lemort, V. Pure and pseudo-pure fluid thermophysical property evaluation and the open-source thermophysical property library coolprop. Ind. Eng. Chem. Res. 53, 2498–2508 (2014).


In [1]:
#Import some libraries
!pip install CoolProp
import numpy as np
import CoolProp.CoolProp as CP
from CoolProp.HumidAirProp import HAPropsSI
from CoolProp.CoolProp import PropsSI
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import scipy.optimize

     |████████████████████████████████| 4.2 MB 4.3 MB/s 


#Define Inputs and Property Functions

In [ ]:
P = 101325  #Assume atmospheric pressure
fluid = "Water"
mixture = 'Air'

def gibbs_HA(T,P,W, T_0):
    # This function calculates the flow exergy of a humid air mixture, on a basis of humid air
    # When T_0 = T, the specific gibbs function is equivalent to the flow exergy.
    # T: Temperature in K
    # P: Pressure in Pa
    # rh: Relative humidity [0,1]
    # T_0: Dead state temperature
    h = HAPropsSI('Hha','T', T, 'P', P, 'W',W)
    s = HAPropsSI('Sha','T', T, 'P', P, 'W',W)
    g = h-T_0*s
    return g

def gibbs_satv(T, T_0):
    # This function calculates the flow exergy of saturated water. 
    # The exergy in the liquid and vapor states are equivalent.
    # When T_0 = T, the specific gibbs function is equivalent to the flow exergy.
    # T: Temperature in K
    # T_0: Dead state temperature
    h = PropsSI('H','T', T, 'Q',0,fluid)
    s = PropsSI('S','T', T, 'Q',0,fluid)
    g = h-T_0*s
    return g

def calcLW(T,P,W,r):
    # This function calculates the minimum LW for water harvesting, using the flow exergy formulation.
    # The limit to infinitesimal recovery is numerically evaluated using a polynomial fit.
    # T: Ambient temperature in K
    # P: Ambient pressure in Pa
    # rh: Ambient relative humidity [0,1]
    # r: initial recovery ratio (for numerical purposes, it doesn't really matter as long as r < 1)
    dw = []
    sep = []
    for i in np.arange(0, 3, 1):
        dw.append(r * W)
        W_reject = W * (1 - r)
        G_p = gibbs_satv(T, T)
        G_f = gibbs_HA(T, P, W, T)
        G_b = gibbs_HA(T, P, W_reject, T)
        sep.append((G_p + G_b * (1 - dw[i]) / dw[i] - G_f / dw[i]) / 1000)
        r = r / 2
        print(r, dw[i], sep[i])
    z = np.polyfit(dw, sep, 2)
    f = np.poly1d(z)
    min_sep = f(0)
    return min_sep

def calcDcooling(T,P,W,r, COP = 3):
    # This function calculates the minimum energy requirement for direct cooling without icing, and the optimal recovery ratio
    # The algorithm finds the maximum recovery to avoid icing. Then it slowly minimizes r until energy is sufficiently minimized
    # T: Ambient temperature in K
    # P: Ambient pressure in Pa
    # rh: Ambient relative humidity [0,1]
    # r: recovery ratio

    alpha = 0.005 #Predefined gradient descent coefficient
    dw = r*W
    W_reject = W * (1 - r)
    T_dp = HAPropsSI('D', 'T', T, 'P', P, 'W', W_reject)
    while T_dp - 273.15 < 5e-4 :  # find the recovery that corresponds to the dew point (avoid freezing)
      r = r*(1-alpha)
      dw = r*W
      W_reject = W * (1 - r)
      T_dp = HAPropsSI('D', 'T', T, 'P', P, 'W', W_reject)
      if (r <=0):
        return('end')
    
    G_p = gibbs_satv(T_dp, T)
    G_f = gibbs_HA(T, P, W, T)
    G_b = gibbs_HA(T_dp, P, W_reject, T)
    sep = (G_p + G_b * (1 - dw) / dw - G_f / dw) / 1000

    sep_new = sep - 1.1e-2  #Small numerical addition to enter the while loop condition
    r_new = r

    while sep - sep_new > 1e-2: # find the energy optimal condition in the constrained range of recovery
      sep  = sep_new
      r = r_new
      r_new = r*(1-alpha)

      dw = r_new*W
      W_reject = W * (1 - r_new)

      T_dp = HAPropsSI('D', 'T', T, 'P', P, 'W', W_reject)

      G_p = gibbs_satv(T_dp, T)
      G_b = gibbs_HA(T_dp, P, W_reject, T)
      sep_new = (G_p + G_b * (1 - dw) / dw - G_f / dw) / 1000
      if r<= 1e-2:
        return r,sep
    return r, sep



    # This function returns the dew harvesting energy requirement as a function of recovery
    # T: Ambient temperature (K)
    # rh: Ambient relative humidity [0,1]

def min_recovery(T,rh):


    recovery = np.linspace(0.01,0.9,20)
    DEW = []
    rec = []
    for r in recovery:
          W = HAPropsSI('W', 'T', T, 'P', P, 'R', rh)
          dw = r*W
          W_reject = W * (1 - r)
          T_dp = HAPropsSI('D', 'T', T, 'P', P, 'W', W_reject)
          if T_dp > 273.1:
            rec.append(r)
            DEW.append(calcDcooling(T,P,W,r))
    loc = np.argmin(DEW)
    min_r = recovery[loc]
    return (min_r)

# Psychometric chart calculation

Note: Complexity scales O^n. For large length (> 20) the program will take > 1 hour on Google colab compute resources. 

The code block below returns a dataframe `df` with columns for temperature `T`, absolute humidity `W`, relative humidity `RH`, minimum least work `LW`, minimum dew work `DC`, and the optimal recovery ratio for dew `RR`.


In [ ]:
length = 150                          # Specifies  the resolution of the contour plot
T = np.linspace(280,325,length)       # Ambient temperature in K
W = np.linspace(0.002,0.04,length)    # Ambient Relative Humidity [0,1]

# Initialize arrays
DC = [] 
recovery = []
T_df = []
relhum = []
abshum = []
r_ref = []
LW = []

# Find the saturation line
W_sat = HAPropsSI('W','T',T,'P',P, 'R',1)

# Loop through all conditions
counter = 0
for hum in W:
    idx = 0
    for temp in T:
        # Check if the dewpoint is above freezing and the mixture is not super saturated
        if (HAPropsSI('D', 'T', temp, 'P', P, 'W', hum)>273.2) & (hum<=W_sat[idx]):
            RH = HAPropsSI('R', 'T', temp, 'P', P, 'W', hum)
            T_dp = HAPropsSI('D', 'T', temp, 'P', P, 'W', hum)
            LW.append(calcLW(temp, P, hum, 0.2))
            # the dew/direct cooling takes a while to run because it uses an optimizer
            #r_temp, dc_temp = calcDcooling(temp,P,hum,0.9)
            #DC.append(dc_temp)
            #recovery.append(r_temp)
            T_df.append(temp)
            relhum.append(RH)
            abshum.append(hum)
            print('temp:', temp, 'humidity:', hum, 'counter', counter, 'index:', idx) 
            counter += 1
        idx += 1

print('Calculations successfully completed')

# Save arrays as a dataframe
data = {'T':T_df,'W': abshum, 'RH':relhum,'LW':LW, 'DC':DC, 'RR':recovery}
df = pd.DataFrame(data)
df.to_csv('recovery_data')

The code block below plots the work for dew harvesting as a function of the recovery ratio. 
 returns a dataframe `df` with columns for temperature `T`, absolute humidity `W`, relative humidity `RH`, minimum least work `LW`, minimum dew work `DC`, and the optimal recovery ratio for dew `RR`.


In [ ]:
fig, ax = plt.subplots(figsize=(5.1667, 5.1667))
T = 300
rh = 0.5
df = recovery_curve(290,0.7)
ax.plot(df['RR'], df['DEW'], 'maroon')